<a href="https://colab.research.google.com/github/marvin-math/active_inference_multiarmed_bandit/blob/main/thesis_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import scipy
from statistics import mean
import random
import pandas as pd

#from scipy.special import psi

# create Bandit class
class BayesianBanditAgent:
    def __init__(self, num_arms, prior_mean, prior_precision, prior_alpha, prior_beta, eta, decay, ucb_weight):
        self.num_arms = num_arms
        self.prior_mean = prior_mean
        self.prior_precision = prior_precision
        self.prior_alpha = prior_alpha
        self.prior_beta = prior_beta
        self.draws = np.zeros(num_arms)
        self.posterior_dist_mean = np.ones(num_arms) # this has to be reworked
        self.posterior_variance = np.ones(num_arms)
        self.posterior_mean = np.ones(num_arms)
        self.posterior_precision = np.ones(num_arms)
        self.posterior_alpha = np.ones(num_arms)
        self.posterior_beta = np.ones(num_arms)
        self.rewards = np.zeros(num_arms)
        self.observations = np.empty(num_arms, dtype=object)
        self.observations[:] = [[] for _ in range(num_arms)]
        self.obs_mean = np.zeros(num_arms)
        self.t_dof = np.zeros(num_arms)
        self.t_mean = np.zeros(num_arms)
        self.t_precision = np.zeros(num_arms)
        self.t_sd = np.zeros(num_arms)
        self.predictive_pdf = np.ones(num_arms)
        self.entropy_predictive_pdf = np.ones(num_arms)
        self.eta = eta
        self.kl_div = np.ones(num_arms)
        self.ambiguity_term = np.ones(num_arms)
        self.expected_free_energy = np.ones(num_arms)
        self.expected_free_energies = np.empty(num_arms, dtype=object)
        self.expected_free_energies[:] = [[] for _ in range(num_arms)]
        self.decay = decay
        self.weight_current = np.ones(num_arms)
        self.moving_avg_previous = np.zeros(num_arms)
        self.weight_previous = np.zeros(num_arms)
        self.moving_avg_current = np.zeros(num_arms)
        self.thompson_mean = np.ones(num_arms)
        self.ucb_weight = ucb_weight
        self.ucb = np.ones(num_arms)
        self.action_values = np.ones(num_arms)
        self.current_reward = np.ones(num_arms)
        self.softmax_probs = np.zeros(num_arms)



    def select_lowest_expected_free_energy_arm(self):
        min_expected_free_energy = float('inf')
        best_arm = 0

        for arm in range(self.num_arms):
            self.expected_free_energy[arm] = self.calc_expected_free_energy(arm)
            self.expected_free_energies[arm].append(self.expected_free_energy[arm])
            #if self.expected_free_energy[arm] <= min_expected_free_energy: # a bit ambiguity here, but probably not relevant
                #min_expected_free_energy = self.expected_free_energy[arm]
                #best_arm = arm
        # Apply softmax function to the expected free energy values
        softmax_probs = self.softmax(self.expected_free_energy)
        self.softmax_probs = softmax_probs
        #print(self.softmax_probs)


        # Choose an arm probabilistically based on the softmax probabilities
        best_arm = np.random.choice(range(self.num_arms), p=softmax_probs)

        self.draws[best_arm] += 1

        return best_arm

    def random_agent(self):
      choice_prob1 = round(random.random(), 3)
      choice_prob2 = round(random.random(), 3)
      self.softmax_probs[0] = choice_prob1 / (choice_prob1 + choice_prob2)
      self.softmax_probs[1] = choice_prob2 / (choice_prob1 + choice_prob2)
      best_arm = np.random.choice(range(self.num_arms), p=self.softmax_probs)
      return best_arm

    def softmax(self, vector):
        # Calculate the softmax values
        exp_vals = np.exp(vector)
        probs = exp_vals / np.sum(exp_vals)
        probs = 1-probs # because we are minimizing
        return probs

    def thompson_sampling(self):
        max_thompson = -float('inf')
        best_arm = 0
        # Draw samples from the posterior distributions
        for arm in range(self.num_arms):
            self.thompson_mean[arm] = np.random.normal(self.posterior_mean[arm], (self.posterior_variance[arm]/np.sqrt(self.posterior_precision[arm])))
            if self.thompson_mean[arm] >= max_thompson:
                max_thompson = self.thompson_mean[arm]
                best_arm = arm
        softmax_probs = self.softmax(self.thompson_mean)
        self.softmax_probs = softmax_probs


        self.draws[best_arm] += 1

        return best_arm


    def ucb_action(self):
        max_ucb = -float('inf')
        best_arm = 0
        for arm in range(self.num_arms):
            # Calculate the uncertainty bonus (UCB) for each arm
            self.ucb[arm] = self.ucb_weight * np.sqrt(1 / self.posterior_precision[arm])

            # Calculate the action values (Q_t(k) + U_t(k)) for each arm
            self.action_values[arm] = self.posterior_mean[arm] + self.ucb[arm]
            #if self.action_values[arm] >= max_ucb:
                # Select the arm with the highest action value
                #max_ucb = self.action_values[arm]
                #best_arm = arm
        softmax_probs = self.softmax(self.action_values)
        self.softmax_probs = softmax_probs
        best_arm = np.random.choice(range(self.num_arms), p=softmax_probs)

        self.draws[best_arm] += 1

        return best_arm

    def update(self, arm, reward):
        # Update the posterior parameters for the selected arm
        self.rewards[arm] += reward
        self.current_reward[arm] = reward
        self.observations[arm].append(reward)
        self.obs_mean[arm] = np.mean(self.observations[arm])

        self.posterior_precision[arm] = self.draws[arm] + self.prior_precision[arm]
        self.posterior_mean[arm] = (self.prior_precision[arm] * self.prior_mean[arm] + self.draws[arm] * self.obs_mean[arm]) / self.posterior_precision[arm]
        self.posterior_alpha[arm] = self.prior_alpha[arm] + 0.5 * self.draws[arm]
        self.posterior_beta[arm] = self.prior_beta[arm] + (0.5 * np.sum((self.observations[arm]-self.obs_mean[arm])**2)) + \
                                    (self.prior_precision[arm] * self.draws[arm]*(self.obs_mean[arm]-self.prior_mean[arm])**2) / (2*(self.posterior_precision[arm]+self.draws[arm]))
        self.posterior_variance[arm] = np.sqrt(np.mean(1/np.random.gamma(self.posterior_alpha[arm], 1/self.posterior_beta[arm], size = 1000))) # this is actually sd
        self.posterior_dist_mean[arm] = mean(np.random.normal(self.posterior_mean[arm], (self.posterior_variance[arm]/np.sqrt(self.posterior_precision[arm])), size= 1000))

        # update the parameters for the next iteration
        self.prior_precision[arm] = self.posterior_precision[arm]
        self.prior_mean[arm] = self.posterior_mean[arm]
        self.prior_alpha[arm] = self.posterior_alpha[arm]
        self.prior_beta[arm] = self.posterior_beta[arm]


    def update_dynamic(self, arm, reward):
        # Update the posterior parameters for the selected arm
        self.rewards[arm] += reward
        self.current_reward[arm] = reward
        self.observations[arm].append(reward)
        self.obs_mean[arm] = np.mean(self.observations[arm])

        # weighting factor calculation
        if len(self.observations[arm]) > 0:
          self.weight_current[arm] = self.decay * self.weight_previous[arm] + 1
        else:
          self.weight_current[arm] = 1
        self.moving_avg_current[arm] = (1 - 1/self.weight_current[arm]) * self.moving_avg_previous[arm] + (1/self.weight_current[arm]) * reward

        # bayesian parameter estimation
        self.posterior_precision[arm] = self.draws[arm] + self.prior_precision[arm]
        self.posterior_mean[arm] = (self.prior_precision[arm] * self.prior_mean[arm] + self.draws[arm] * self.moving_avg_current[arm]) / self.posterior_precision[arm]
        self.posterior_alpha[arm] = self.prior_alpha[arm] + 0.5 * self.draws[arm]
        self.posterior_beta[arm] = self.prior_beta[arm] + (0.5 * np.sum((self.observations[arm]-self.moving_avg_current[arm])**2)) + \
                                    (self.prior_precision[arm] * self.draws[arm]*(self.moving_avg_current[arm]-self.prior_mean[arm])**2) / (2*(self.posterior_precision[arm]+self.draws[arm]))
        self.posterior_variance[arm] = np.sqrt(np.mean(1/np.random.gamma(self.posterior_alpha[arm], 1/self.posterior_beta[arm], size = 1000))) # this is actually sd
        self.posterior_dist_mean[arm] = mean(np.random.normal(self.posterior_mean[arm], (self.posterior_variance[arm]/np.sqrt(self.posterior_precision[arm])), size = 1000))

        # update the parameters for the next iteration
        self.prior_precision[arm] = self.posterior_precision[arm]
        self.prior_mean[arm] = self.posterior_mean[arm]
        self.prior_alpha[arm] = self.posterior_alpha[arm]
        self.prior_beta[arm] = self.posterior_beta[arm]

        # update weighting factor parameters for next iteration
        self.weight_previous[arm] = self.weight_current[arm]
        self.moving_avg_previous[arm] = self.moving_avg_current[arm]


    def posterior_predictive(self, arm):
        # Calculate the posterior predictive t-distribution parameters for the selected arm
        self.t_dof[arm] = 2*(self.posterior_alpha[arm] + 1)
        self.t_mean[arm] = self.posterior_mean[arm]
        self.t_precision[arm] = (self.posterior_beta[arm] * (1 + self.posterior_precision[arm])) / (self.posterior_precision[arm]* (1 + self.posterior_alpha[arm]))
        self.t_sd[arm] = 1/np.sqrt(self.t_precision[arm]) # scipy uses standard deviation, so we have to transform


        v = self.t_dof[arm]
        v1 = (1+v)/2
        v2 = v/2
        self.entropy_predictive_pdf[arm] = v1*(scipy.special.digamma(v1)-scipy.special.digamma(v2))+np.log(np.sqrt(v)*scipy.special.beta(v2,0.5))

        return self.entropy_predictive_pdf[arm]

    def kl_divergence(self, arm, eta):
        entropy_of_predictive = self.posterior_predictive(arm)
        self.kl_div[arm] = - eta*self.posterior_mean[arm] - entropy_of_predictive
        return self.kl_div[arm]

    def ambiguity(self, arm):
        self.ambiguity_term[arm] = 0.5 * np.log(2 * np.pi * np.e) + np.log(self.posterior_beta[arm]) + scipy.special.digamma(self.posterior_alpha[arm])
        return self.ambiguity_term[arm]

    def calc_expected_free_energy(self, arm):
        self.expected_free_energy_term = self.kl_divergence(arm, eta = self.eta) + self.ambiguity(arm)
        return self.expected_free_energy_term

    def optimal_strategy(self, true_arm_means):
        return np.argmax(true_arm_means)









In [ ]:
# import the data
url = 'https://raw.githubusercontent.com/marvin-math/active_inference_multiarmed_bandit/main/musexp1.csv'
df1 = pd.read_csv(url)
np_array1_exp1 = df1['mu1'].to_numpy()
np_array2_exp1 = df1['mu2'].to_numpy()

url2 = 'https://raw.githubusercontent.com/marvin-math/active_inference_multiarmed_bandit/main/exp2data.csv'
df2 = pd.read_csv(url2)
np_array1_exp2 = df2['mu1'].to_numpy()
np_array2_exp2 = df2['mu2'].to_numpy()
np_arraychoice_exp2 = df2['choice'].to_numpy()
np_arrayreward_exp2 = df2['reward'].to_numpy()
np_arraysubject_exp2 = df2['subject'].to_numpy()

url3 ='https://raw.githubusercontent.com/marvin-math/active_inference_multiarmed_bandit/main/optimization_ai.csv'
df3 = pd.read_csv(url3)
np_eta = df3['eta'].to_numpy()
np_decay = df3['decay'].to_numpy()

url4 ='https://raw.githubusercontent.com/marvin-math/active_inference_multiarmed_bandit/main/optimization_ucb.csv'
df4 = pd.read_csv(url4)
np_ucb = df4['ucb'].to_numpy()
#np_decay = df4['decay'].to_numpy()

url5 ='https://raw.githubusercontent.com/marvin-math/active_inference_multiarmed_bandit/main/df_thompson.csv'
df5 = pd.read_csv(url5)
#np_decay = df5['decay'].to_numpy()

url6 ='https://raw.githubusercontent.com/marvin-math/active_inference_multiarmed_bandit/main/df_optimization_random.csv'
df6 = pd.read_csv(url6)
#np_decay = df6['decay'].to_numpy()

In [ ]:
from pickle import TRUE
# initialise

num_arms = 2

num_total_iterations = len(np_array1_exp2)

trials = np.zeros(num_total_iterations)
expected_free_energy_arm0 = np.zeros(num_total_iterations)
expected_free_energy_arm1 = np.zeros(num_total_iterations)
posterior_dist_mean_arm0 = np.zeros(num_total_iterations)
posterior_dist_mean_arm1 = np.zeros(num_total_iterations)
posterior_variance_arm0 = np.zeros(num_total_iterations)
posterior_variance_arm1 = np.zeros(num_total_iterations)
posterior_mean_arm0 = np.zeros(num_total_iterations)
posterior_mean_arm1 = np.zeros(num_total_iterations)
posterior_precision_arm0 = np.zeros(num_total_iterations)
posterior_precision_arm1 = np.zeros(num_total_iterations)
posterior_alpha_arm0 = np.zeros(num_total_iterations)
posterior_alpha_arm1 = np.zeros(num_total_iterations)
posterior_beta_arm0 = np.zeros(num_total_iterations)
posterior_beta_arm1 = np.zeros(num_total_iterations)
rewards = np.zeros(num_total_iterations)
cumulative_rewards = np.zeros(num_total_iterations)
cumulative_optimal_rewards = np.zeros(num_total_iterations)
thompson_mean_arm0 = np.zeros(num_total_iterations)
thompson_mean_arm1 = np.zeros(num_total_iterations)
ucb_arm0 = np.zeros(num_total_iterations)
ucb_arm1 = np.zeros(num_total_iterations)
action_values_arm0 = np.zeros(num_total_iterations)
action_values_arm1 = np.zeros(num_total_iterations)
chosen_arms = np.zeros(num_total_iterations)
optimal_arms = np.zeros(num_total_iterations)
optimal_rewards = np.zeros(num_total_iterations)
regrets = np.zeros(num_total_iterations)
cumulative_regrets = np.zeros(num_total_iterations)
etas_list = np.zeros(num_total_iterations)
decays_list = np.zeros(num_total_iterations)
true_dist_mean0 = np.zeros(num_total_iterations)
true_dist_mean1 = np.zeros(num_total_iterations)
draws0 = np.zeros(num_total_iterations)
draws1 = np.zeros(num_total_iterations)
ucb_list = np.zeros(num_total_iterations)
ambiguity_arm0 = np.zeros(num_total_iterations)
ambiguity_arm1 = np.zeros(num_total_iterations)

num_simulations = 1
num_iterations_per_simulation = len(np_array1_exp2)

true_arm_means_dynamic = np.empty(num_arms, dtype=object)
true_arm_means_dynamic[:] = [[] for _ in range(num_arms)]





#setup experiment environment
environment = 'dynamic'
experiment1 = False


#eta = 0.3 # exploration exploitation parameter - this parameter gives the weight the posterior mean has in the decision (higher weight -> lower kl divergence -> lower expected free energy)
#decay = 0.1 # how much are previous samples taken into account? Between 0 and 1 - higher values give more weight to older data, lower values more weight to current data
ucb_weight = 0.6

#create empty dict to store the results
result_df = pd.DataFrame(columns=['trial', 'expected_free_energy', 'posterior_dist_mean', 'posterior_variance', 'posterior_mean', 'posterior_precision', 'posterior_alpha', 'posterior_beta', 'reward', 'cumulative_reward', 'cumulative_optimal_reward', 'thompson_mean', 'ucb', 'action_values', 'chosen_arm', 'optimal_arm', 'optimal_reward', 'regret', 'cumulative_regret', 'true_dist_mean', 'draw_sd', 'decay', 'ucb_weight', 'eta', 'draws0', 'draws1', 'ambiguity_arm0', 'ambiguity_arm1'])
index = 0
# Reset cumulative variables for each simulation
cumulative_reward = 0
cumulative_regret = 0
cumulative_optimal_reward = 0

for participant in range(44):
    mean_of_true_rewards = 5
    mean_of_true_sd = 3
    prior_mean = np.ones(num_arms)
    prior_precision = np.ones(num_arms)
    prior_alpha = np.ones(num_arms)
    prior_beta = np.ones(num_arms)*2

    mu1 = np_array1_exp2[np_arraysubject_exp2 == (participant+1)]
    mu2 = np_array2_exp2[np_arraysubject_exp2 == (participant+1)]
    print(participant)
    eta = np_eta[participant]
    decay = np_decay[participant]
    #ucb_weight = np_ucb[participant]
    #eta = 0.3 # dummy


    # Create the Bayesian bandit agent
    agent = BayesianBanditAgent(num_arms, prior_mean, prior_precision, prior_alpha, prior_beta, eta, decay, ucb_weight)

    # Simulate the multi-armed bandit problem for a certain number of iterations
    if environment == 'stationary':
      num_iterations = 5000
      arm_means =  [3, 10]      #np.random.normal(mean_of_true_rewards, 1.0, num_arms)  # True mean rewards for each arm
      arm_sds = [10, 2]      #np.random.gamma(mean_of_true_sd, 2.0, num_arms) # true sd for each arm


    elif environment == 'dynamic':
      if experiment1 == True:
        true_arm_means_dynamic[0] = np_array1_exp1
        true_arm_means_dynamic[1] = np_array2_exp1

      elif experiment1 == False:
        true_arm_means_dynamic[0] = np_array1_exp2
        true_arm_means_dynamic[1] = np_array2_exp2
      num_iterations = len(true_arm_means_dynamic[0])



    for t in range(len(mu1)):
      # Select an arm
      chosen_arm = agent.select_lowest_expected_free_energy_arm()

      if environment == 'stationary':

        # Simulate the reward for the chosen arm (normally distributed)
        reward = np.random.normal(arm_means[chosen_arm], arm_sds[chosen_arm])
        cumulative_reward += reward

        # Update the agent with the observed reward
        agent.update_dynamic(chosen_arm, reward)
      elif environment == 'dynamic':
        true_sd = np.sqrt(10)
        if experiment1 == True:
          if chosen_arm == 0:
            reward = np.random.normal(mu1[t], true_sd)
          elif chosen_arm == 1:
            reward = 0
        elif experiment1 == False:
          if chosen_arm == 0:
            reward = np.random.normal(mu1[t], true_sd)
          elif chosen_arm == 1:
            reward = np.random.normal(mu2[t], true_sd)


        cumulative_reward += reward
        trial_number = t+1

        agent.update_dynamic(chosen_arm, reward)
        # Append values to the numpy array after each trial
        true_arm_means = [mu1[t], mu2[t]]
        optimal_arm = agent.optimal_strategy(true_arm_means)
        optimal_reward = np.random.normal(true_arm_means[optimal_arm], true_sd)
        cumulative_optimal_reward += optimal_reward
        regret = optimal_reward - reward
        cumulative_regret += regret

        trials[index] = t + 1

        # Update the values for both arms after each trial
        chosen_arms[index] = chosen_arm
        optimal_arms[index] = optimal_arm
        optimal_rewards[index] = optimal_reward
        regrets[index] = regret
        cumulative_regrets[index] = cumulative_regret
        etas_list[index] = eta
        decays_list[index] = decay
        ucb_list[index] = ucb_weight
        expected_free_energy_arm0[index] = agent.calc_expected_free_energy(0)
        expected_free_energy_arm1[index] = agent.calc_expected_free_energy(1)
        posterior_dist_mean_arm0[index] = agent.posterior_dist_mean[0]
        posterior_dist_mean_arm1[index] = agent.posterior_dist_mean[1]
        posterior_variance_arm0[index] = agent.posterior_variance[0]
        posterior_variance_arm1[index] = agent.posterior_variance[1]
        posterior_mean_arm0[index] = agent.posterior_mean[0]
        posterior_mean_arm1[index] = agent.posterior_mean[1]
        posterior_precision_arm0[index] = agent.posterior_precision[0]
        posterior_precision_arm1[index] = agent.posterior_precision[1]
        posterior_alpha_arm0[index] = agent.posterior_alpha[0]
        posterior_alpha_arm1[index] = agent.posterior_alpha[1]
        posterior_beta_arm0[index] = agent.posterior_beta[0]
        posterior_beta_arm1[index] = agent.posterior_beta[1]
        rewards[index] = reward
        cumulative_rewards[index] = cumulative_reward
        cumulative_optimal_rewards[index] = cumulative_optimal_reward
        thompson_mean_arm0[index] = agent.thompson_mean[0]
        thompson_mean_arm1[index] = agent.thompson_mean[1]
        ucb_arm0[index] = agent.ucb[0]
        ucb_arm1[index] = agent.ucb[1]
        action_values_arm0[index] = agent.action_values[0]
        action_values_arm1[index] = agent.action_values[1]
        true_dist_mean0[index] = true_arm_means[0]
        true_dist_mean1[index] = true_arm_means[1]
        draws0[index] = agent.draws[0]
        draws1[index] = agent.draws[1]
        ambiguity_arm0[index] = agent.ambiguity_term[0]
        ambiguity_arm1[index] = agent.ambiguity_term[1]




        index += 1

# Create a pandas DataFrame with the collected data
result_df = pd.DataFrame({
    'trial': trials,
    'eta': etas_list,
    'decay': decays_list,
    'ucb': ucb_list,
    'expected_free_energy_arm0': expected_free_energy_arm0,
    'expected_free_energy_arm1': expected_free_energy_arm1,
    'posterior_dist_mean_arm0': posterior_dist_mean_arm0,
    'posterior_dist_mean_arm1': posterior_dist_mean_arm1,
    'posterior_variance_arm0': posterior_variance_arm0,
    'posterior_variance_arm1': posterior_variance_arm1,
    'posterior_mean_arm0': posterior_mean_arm0,
    'posterior_mean_arm1': posterior_mean_arm1,
    'posterior_precision_arm0': posterior_precision_arm0,
    'posterior_precision_arm1': posterior_precision_arm1,
    'posterior_alpha_arm0': posterior_alpha_arm0,
    'posterior_alpha_arm1': posterior_alpha_arm1,
    'posterior_beta_arm0': posterior_beta_arm0,
    'posterior_beta_arm1': posterior_beta_arm1,
    'rewards': rewards,
    'cumulative_rewards': cumulative_rewards,
    'cumulative_optimal_rewards': cumulative_optimal_rewards,
    'thompson_mean_arm0': thompson_mean_arm0,
    'thompson_mean_arm1': thompson_mean_arm1,
    'ucb_arm0': ucb_arm0,
    'ucb_arm1': ucb_arm1,
    'action_values_arm0': action_values_arm0,
    'action_values_arm1': action_values_arm1,
    'chosen_arm': chosen_arms,
    'optimal_arm': optimal_arms,
    'optimal_reward': optimal_rewards,
    'regret': regrets,
    'cumulative_regret': cumulative_regrets,
    'true_dist_mean0': true_dist_mean0,
    'true_dist_mean1': true_dist_mean1,
    'draw_sd': np.sqrt(10),
    'draws0': draws0,
    'draws1': draws1,
    'ambiguity_arm0': ambiguity_arm0,
    'ambiguity_arm1': ambiguity_arm1
})









result_df.to_csv('/content/simulation_results_ai.csv', index=False)



#print("Posterior Predictive PDF for New Data Point:", posterior_predictive_pdf)
print(f"the posterior means are: {np.mean(agent.posterior_dist_mean)}")
print(f"the posterior sd is: {np.mean(agent.posterior_variance)}")
#print(f"the true arm means are: {np.mean(true_arm_means_dynamic[0])} and {np.mean(true_arm_means_dynamic[1])}")
print(f"the true arm sds are: {np.sqrt(10)}")
print(f"the cumulative reward is: {cumulative_reward}")

#print(f"entropy of t-distribution is: {entropy}")
#print(f"kl_divergence is: {agent.kl_divergence}")
#print(f"ambiguity is: {agent.ambiguity}")
print(f"the expected free energy of arm 1: {np.mean(expected_free_energy_arm0)} and arm 2: {np.mean(expected_free_energy_arm1)}")
#print(f"draws: {agent.draws}")
#print(f"kl divergences: {np.mean(agent.kl_div)}")
print(f"total draws: {agent.draws[0] + agent.draws[1]}")
print(f"the entropies of the predictives are: {np.mean(agent.entropy_predictive_pdf[0])} and {np.mean(agent.entropy_predictive_pdf[1])}")
print(f"the kl divergences are: {np.mean(agent.kl_div[0])} and {np.mean(agent.kl_div[1])}")
print(f"the ambiguity terms are: {np.mean(agent.ambiguity_term[0])} and {np.mean(agent.ambiguity_term[1])}")


for arm in range(num_arms):
  #print(f"mean free energy per arm: {np.mean(agent.expected_free_energies[arm])}")
  print(f"arm {arm} has been chosen {int(agent.draws[arm])} times")
  #print(f"observation matrix: {agent.observations}")









In [ ]:
from scipy.optimize import Bounds, minimize

# initialise dataframe to save the values

results_df = pd.DataFrame(columns=['Participant', 'eta', 'decay', 'Optimal Likelihood'])

# this section runs the log likelihood minimzation algorithm
num_arms = 2
num_iterations = len(np_arrayreward_exp2)
p = np.empty(num_arms, dtype=object)
p[:] = [[] for _ in range(num_arms)]

# Define the objective function with proper argument structure
def neg_fe_log_lik(parameters, *args):
    ucb_weight = 0.6
    eta = parameters[0]
    decay = parameters[1]
    prior_mean = np.ones(num_arms) * 0
    prior_precision = np.ones(num_arms)
    prior_alpha = np.ones(num_arms)
    prior_beta = np.ones(num_arms) * 2
    negloglikelihood = 0

    agent = BayesianBanditAgent(num_arms, prior_mean, prior_precision, prior_alpha, prior_beta, eta, decay, ucb_weight)



    for t in range(len(participant_choices)):
        #best_arm = agent.select_lowest_expected_free_energy_arm()
        #best_arm = agent.thompson_sampling()
        best_arm = agent.select_lowest_expected_free_energy_arm()
        if participant_choices[t] == 0:
            negloglikelihood += np.log(agent.softmax_probs[0]) # the posterior variance is the posterior of previour trial, so technically the prior of this trial
        else:
            negloglikelihood += np.log(agent.softmax_probs[1])
        agent.update_dynamic(best_arm, participant_rewards[t])
        #print(agent.softmax_probs)


    negloglikelihood = (-negloglikelihood)
    print(parameters, negloglikelihood)
    return negloglikelihood

# initial eta and decay setting
initial_parameters = [0.7, 0.3] # 0.3 #

# Define bounds for each parameter
#ucb_bounds = (0, 2.5758)  # Example bounds for eta parameter
decay_bounds = (0, 1)  # Example bounds for decay parameter
eta_bounds = (0, 1)

# Create a Bounds object
parameter_bounds = Bounds([eta_bounds[0], decay_bounds[0]], [eta_bounds[1],decay_bounds[1]])

# Iterate through participants and fit parameters separately
for participant in range(44):
    participant_choices = np_arraychoice_exp2[np_arraysubject_exp2 == (participant+1)]
    participant_rewards = np_arrayreward_exp2[np_arraysubject_exp2 == (participant+1)]
    print(participant)


    result = minimize(
        neg_fe_log_lik,
        initial_parameters, method = 'Powell', options={'xtol':1e-3, 'ftol': 1e-3},
        args=(participant_choices, participant_rewards),
        bounds=parameter_bounds
    )

        # Append results to the DataFrame
    results_df = results_df.append({
        'Participant': participant+1,
        'eta': result.x[0],
        'decay': result.x[1],
        'Optimal Likelihood': result.fun
    }, ignore_index=True)

results_df.to_csv('optimization_ai.csv', index=False)


"""    print("Optimization Result for Participant", participant + 1)
    print("Optimal Parameters:", result.x)
    print("Optimal Value of the Objective Function:", result.fun)"""



In [ ]:
print(result)

 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: 262.3486128551278
       x: [ 1.246e-01]
     nit: 3
   direc: [[ 1.000e+00]]
    nfev: 42
